In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

In [2]:
def get_bits(x, n):
    return [int(x) for x in '{:0{size}b}'.format(x, size=n)]

In [3]:
def generate_carry(constant, n):
    name = 'Carry ({0})'.format(constant)
    constant_bits = get_bits(constant, n)
    
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrA = QuantumRegister(n, name='a')
    qrC = QuantumRegister(1, name='c')
    
    # special case for n = 1: 
    if n == 1:
        qc = QuantumCircuit(qrCtrl, qrA, qrC, name=name)
    
        if constant_bits[0] == 1:
            qc.ccx(qrCtrl[0], qrA[0], qrC)
        
        return qc.to_instruction()
    
    # for n >= 2:
    qrG = QuantumRegister(n-1, name='g')
    
    qc = QuantumCircuit(qrCtrl, qrA, qrG, qrC, name=name)

    qc.ccx(qrCtrl[0], qrG[n - 2], qrC)

    for i in range(n - 1, 1, -1):
        if constant_bits[n - (i + 1)] == 1:
            qc.cx(qrA[i], qrG[i - 1])
            qc.x(qrA[i])
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])

    if constant_bits[n - 2] == 1:
        qc.cx(qrA[1], qrG[0])
        qc.x(qrA[1])
        
    if constant_bits[n - 1] == 1:
        qc.ccx(qrA[0], qrA[1], qrG[0])

    for i in range(2, n):
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])

    qc.ccx(qrCtrl[0], qrG[n - 2], qrC)
   
    for i in range(n - 1, 1, -1):
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])

    if constant_bits[n - 1] == 1:
        qc.ccx(qrA[0], qrA[1], qrG[0])
        
    if constant_bits[n - 2] == 1:
        qc.x(qrA[1])
        qc.cx(qrA[1], qrG[0])

    for i in range(2, n):
        qc.ccx(qrG[i - 2], qrA[i], qrG[i - 1])
        if constant_bits[n - (i + 1)] == 1:
            qc.x(qrA[i])
            qc.cx(qrA[i], qrG[i - 1])
    
    return qc.to_instruction()

def generate_substractor(n):
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrX = QuantumRegister(n, name='g')
    qrY = QuantumRegister(n, name='a')
        
    qc = QuantumCircuit(qrCtrl, qrY, qrX)
    
    # 3.6
    for i in range(n-1, 0, -1):
        qc.cx(qrX[i], qrY[i])
        
    qc.ccx(qrCtrl[0], qrX[0], qrY[0])
    
    # 3.5
    for i in range(n-1, 1, -1):
        qc.cx(qrX[i-1], qrX[i])

    # 3.4
    for i in range(1, n):
        qc.ccx(qrX[i-1], qrY[i-1], qrX[i])
        
        qc.ccx(qrCtrl[0], qrX[i], qrY[i])

    # 3.3
    for i in range(n-1, 0, -1):
        qc.ccx(qrX[i-1], qrY[i-1], qrX[i])

    # 3.2
    for i in range(1, n-1):
        qc.cx(qrX[i], qrX[i+1])

    # 3.1
    for i in range(n-1, 0, -1):
        qc.cx(qrX[i], qrY[i])
        
    return qc

def generate_negation_part(n):
    qrG = QuantumRegister(n, name='g')
    
    qc = QuantumCircuit(qrG)
    
    qc.barrier(qrG[:])
    for i in range(n):
        qc.x(qrG[i])
    qc.barrier(qrG[:])
    
    return qc

def generate_incrementator(n):
    qc_sub = generate_substractor(n)
    qc_neg = generate_negation_part(n)
    
    qc = qc_sub + qc_neg + qc_sub + qc_neg
    qc.name = 'Inc'
    return qc.to_instruction()

def generate_controlled_negation(n):
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrG = QuantumRegister(n, name='g')
    
    qc = QuantumCircuit(qrCtrl, qrG)
    
    qc.barrier(qrG[:])
    for i in range(n):
        qc.cx(qrCtrl[0], qrG[i])
    qc.barrier(qrG[:])
    
    return qc.to_instruction()

def generate_constant_adder(constant, n):
    name = 'Add ({0})'.format(constant)
    
    qrCtrl = QuantumRegister(1, 'ctrl')
    qrX = QuantumRegister(n, 'x')
    qrC = QuantumRegister(1, 'carry')
    
    qc = QuantumCircuit(qrCtrl, qrX, qrC, name=name)
    
    if n == 1:
        if constant == 1:
            qc.cx(qrCtrl[0], qrX[0])
        return qc.to_instruction()
    
    i = n // 2 + n % 2
    
    low = constant & ((1 << i) - 1)
    high = constant >> i
    
    qrX_L = qrX[:i]
    qrX_H = qrX[i:]
    
    carry = generate_carry(low, i)
    inc = generate_incrementator(n - i)
    neg = generate_controlled_negation(n - i)
    
    qc.append(inc, qrC[:] + qrX_H[:] + qrX_L[:len(qrX_H)])
    qc.append(neg, qrC[:] + qrX_H[:])
    
    qc.append(carry, qrCtrl[:] + qrX_L[:] + qrX_H[:(i-1)] + qrC[:])
    qc.append(inc, qrC[:] + qrX_H[:] + qrX_L[:len(qrX_H)])
    qc.append(carry, qrCtrl[:] + qrX_L[:] + qrX_H[:(i-1)] + qrC[:])
    
    qc.append(neg, qrC[:] + qrX_H[:])
    
    qc.append(generate_constant_adder(low, i), qrCtrl[:] + qrX[:i] + qrC[:])
    qc.append(generate_constant_adder(high, n - i), qrCtrl[:] + qrX[i:] + qrC[:])
    
    return qc.to_instruction()

def generate_constant_substractor(constant, n):
    adder = generate_constant_adder(constant, n)
    return adder.inverse()

In [4]:
def generate_negation(n):
    qrA = QuantumRegister(n, name='a')
    
    qc = QuantumCircuit(qrA, name='Negation')
    
    for i in range(n):
        qc.x(qrA[i])
    
    return qc.to_instruction()

def generate_comparator(constant, n):
    name = 'Comp ({0})'.format(constant)
    
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrA = QuantumRegister(n, name='a')
    qrC = QuantumRegister(1, name='cmp')
    
    carry = generate_carry(constant, n)
    neg = generate_negation(n)
    
    if n == 1:
        qc = QuantumCircuit(qrCtrl, qrA, qrC, name=name)
        qc.append(neg, qrA[:])
        qc.append(carry, qrCtrl[:] + qrA[:] + qrC[:])
        qc.append(neg, qrA[:])
        return qc.to_instruction()

    qrG = QuantumRegister(n-1, name='g')
    qc = QuantumCircuit(qrCtrl, qrA, qrG, qrC, name=name)
    
    qc.append(neg, qrA[:])
    qc.append(carry, qrCtrl[:] + qrA[:] + qrG[:] + qrC[:])
    qc.append(neg, qrA[:])
    
    return qc.to_instruction()

In [5]:
def generate_constant_modulo_adder(constant, N, n):
    name = 'Add({0}) Mod {1}'.format(constant, N)
    
    qrCtrl = QuantumRegister(1, 'ctrl')
    qrX = QuantumRegister(n, 'x')
    qrFlag = QuantumRegister(1, 'flag')

    cmp1 = generate_comparator(N - constant, n)
    adder = generate_constant_adder(constant, n)
    substractor = generate_constant_substractor(N - constant, n)
    cmp2 = generate_comparator(constant, n)
    
    if n == 1:
        qrG = QuantumRegister(1, 'g')
        qc = QuantumCircuit(qrCtrl, qrX, qrG, qrFlag, name=name)
        qc.append(cmp1, [qrCtrl[0], qrX[0], qrFlag[0]])
        qc.append(adder, [qrFlag[0], qrX[0], qrG[0]])
        qc.cx(qrCtrl[0], qrFlag[0])
        qc.append(substractor, [qrFlag[0], qrX[0], qrG[0]])
#         qc.cx(qrCtrl[0], qrFlag[0])
        qc.append(cmp2, [qrCtrl[0], qrX[0], qrFlag[0]])
        
        return qc.to_instruction()
    
    qrG = QuantumRegister(n-1, 'g')
    qc = QuantumCircuit(qrCtrl, qrX, qrG, qrFlag, name=name)
    
    qc.append(cmp1, qrCtrl[:] + qrX[:] + qrG[:] + qrFlag[:])
    qc.append(adder, qrFlag[:] + qrX[:] + [qrG[0]])
    qc.cx(qrCtrl[0], qrFlag[0])
    qc.append(substractor, qrFlag[:] + qrX[:] + [qrG[0]])
#     qc.cx(qrCtrl[0], qrFlag[0])
    qc.append(cmp2, qrCtrl[:] + qrX[:] + qrG[:] + qrFlag[:])
    
    return qc.to_instruction()

In [6]:
# _qrCtrl = QuantumRegister(1, name='ctrl')
# _qrA = QuantumRegister(5, name='a')
# _qrG = QuantumRegister(4, name='g')
# _qrF = QuantumRegister(1, name='flag')
# _qc = QuantumCircuit(_qrCtrl, _qrA, _qrG, _qrF)

# caddmod = generate_constant_modulo_adder(21, 31, 5)
# _qc.append(caddmod, _qrCtrl[:] + _qrA[:] + _qrG[:] + _qrF[:])
# _qc.draw(output='mpl')

In [7]:
# _dec = _qc.decompose()
# _dec.draw(output='mpl')

In [8]:
def generate_init_part(ctrl, x, g, n):
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrX = QuantumRegister(n, name='x')
    
    if n == 1:
        qc = QuantumCircuit(qrCtrl, qrX)
    else:
        qrG = QuantumRegister(n-1, name='g')
        qc = QuantumCircuit(qrCtrl, qrX, qrG)
      
    if ctrl:
        qc.x(qrCtrl[0])
        
    x_bits = get_bits(x, n)
    for i in range(n):
        if x_bits[n-(i+1)] == 1:
            qc.x(qrX[i])
                
    if n == 1:
        qc.barrier(qrCtrl[:], qrX[:])
    else:
        g_bits = get_bits(g, n-1)
        for i in range(n-1):
            if g_bits[n-1 -(i+1)] == 1:
                qc.x(qrG[i])

        qc.barrier(qrCtrl[:], qrX[:], qrG[:])
        
    return qc

In [9]:
# _qcInit = generate_init_part(True, 10, 5, 4)
# _qcInit.draw(output='mpl')

In [10]:
def generate_measure_part(n):
    qrX = QuantumRegister(n, name='x')
    qrC = QuantumRegister(1, name='cmp')
    crX = ClassicalRegister(n, name='xValue')
    crC = ClassicalRegister(1, name='cValue')
    
    if n == 1:
        qc = QuantumCircuit(qrX, qrC, crX, crC)
        qc.barrier(qrX[:], qrC[:])
    else:
        qrG = QuantumRegister(n - 1, name='g')
        crG = ClassicalRegister(n - 1, name='gValue')
    
        qc = QuantumCircuit(qrX, qrG, qrC, crX, crG, crC)
        qc.barrier(qrX[:], qrG[:], qrC[:])
        
    qc.measure(qrX[:], crX[:])
    
    if n > 1:
        qc.measure(qrG[:], crG[:])
    
    qc.measure(qrC[:], crC[:])
    
    return qc

In [11]:
# _qcMeas = generate_measure_part(4)
# _qcMeas.draw(output='mpl')

In [12]:
def test_constant_modulo_addition(ctrl, x, constant, N, value):
    expected_value = (x + constant) % N if ctrl else x
    print(expected_value, value, expected_value == value)

In [13]:
test_constant_modulo_addition(True, 14, 21, 31, 4)
test_constant_modulo_addition(False, 14, 21, 31, 14)

4 4 True
14 14 True


In [14]:
from qiskit import Aer
from qiskit import execute

backend = Aer.get_backend('qasm_simulator')

In [15]:
def test(ctrl, x, constant, N, g, n):
    qrCtrl = QuantumRegister(1, name='ctrl')
    qrX = QuantumRegister(n, name='x')
    qrC = QuantumRegister(1, 'cmp')
    
    if n == 1:
        qrG = QuantumRegister(1, name='g')
        qc = QuantumCircuit(qrCtrl, qrX, qrG, qrC)
    else:
        qrG = QuantumRegister(n-1, name='g')
        qc = QuantumCircuit(qrCtrl, qrX, qrG, qrC)
        
    qc += generate_init_part(ctrl, x, g, n)
    
    qc.append(generate_constant_modulo_adder(constant, N, n), qrCtrl[:] + qrX[:] + qrG[:] + qrC[:])
        
    qc += generate_measure_part(n)

#     return qc
    job = execute(qc, backend, shots=1)
    result = job.result()
    values = list(result.get_counts(qc).keys())

    assert len(values) == 1
    values = [int(v, 2) for v in values[0].split(' ')]

    if n == 1:
        c_value = values[0]
        x_value = values[1]

        print(0, c_value, 0 == c_value)
        test_constant_modulo_addition(ctrl, x, constant, N, x_value)
    else:
        c_value = values[0]
        g_value = values[1]
        x_value = values[2]

        print(g, g_value, g == g_value)
        print(0, c_value, 0 == c_value)
        test_constant_modulo_addition(ctrl, x, constant, N, x_value)
        
#     return qc

In [16]:
_qc = test(True, 6, 21, 31, 15, 5)

<ipython-input-15-cce89bc839ef>:13: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  qc += generate_init_part(ctrl, x, g, n)
/home/bartek/S10/MGR/masterThesisProject/venv/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:876: DeprecationWarning: The QuantumCircuit.extend() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  return self.extend(rhs)
<ipython-input-3-2d40546e6340>:114: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  qc = qc_sub + qc_neg + qc_sub + qc_neg
/home/bartek/S10/MGR/masterThesisProject/venv/lib/python3.8/site-packages/qiskit/circuit/quantumcir

15 15 True
0 0 True
27 27 True


In [17]:
# _dec = _qc.decompose()
# _dec.draw(output='mpl')

In [18]:
# for n in range(5):
#     n = n + 1
#     print(n)
#     G = 2 ** (n-1)
    
#     for N in range(1, 2 ** n):
#         for x in range(N):
#             for g in range(G):
#                 for c in range(N):
#                     print('---', 0, '-', x, c, N, g, '---')
#                     test(False, x, c, N, g, n)

In [19]:
# for n in range(5):
#     n = n + 1
#     print(n)
#     G = 2 ** (n-1)
    
#     for N in range(1, 2 ** n):
#         for x in range(N):
#             for g in range(G):
#                 for c in range(N):
#                     print('---', 1, '-', x, c, N, g, '---')
#                     test(True, x, c, N, g, n)